In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/hackthechain/dataset.csv
/kaggle/input/gemma/keras/gemma_instruct_2b_en/2/config.json
/kaggle/input/gemma/keras/gemma_instruct_2b_en/2/tokenizer.json
/kaggle/input/gemma/keras/gemma_instruct_2b_en/2/metadata.json
/kaggle/input/gemma/keras/gemma_instruct_2b_en/2/model.weights.h5
/kaggle/input/gemma/keras/gemma_instruct_2b_en/2/assets/tokenizer/vocabulary.spm
/kaggle/input/chatbot/dataset.txt


In [2]:
df = pd.read_csv('/kaggle/input/hackthechain/dataset.csv')

In [3]:
# !pip install wurlitzer
# !pip install -q -U keras-nlp
# !pip install -q -U keras>=3

In [4]:
import os

os.environ["KERAS_BACKEND"] = "jax"  # Or "torch" or "tensorflow".
os.environ["XLA_PYTHON_CLIENT_MEM_FRACTION"]="1.00"

In [5]:
import keras
import keras_nlp
gemma_lm = keras_nlp.models.GemmaCausalLM.from_preset("gemma_instruct_2b_en")

normalizer.cc(51) LOG(INFO) precompiled_charsmap is empty. use identity normalization.


In [6]:
data = []
for index, row in df.iterrows():
    
    question = row['Question']
    answer = row['Answer']
    template = "Instruction:\n{instruction}\n\nResponse:\n{response}"
    formatted_example = template.format(instruction=question, response=answer)

    data.append(formatted_example)

In [7]:

gemma_lm.backbone.enable_lora(rank=64)

In [8]:
gemma_lm.preprocessor.sequence_length = 314

optimizer = keras.optimizers.AdamW(
    learning_rate=5e-5,
    weight_decay=0.01,
    beta_1=0.9,         
    beta_2=0.999         
)

optimizer.exclude_from_weight_decay(var_names=["bias", "scale"])

gemma_lm.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=optimizer,
    weighted_metrics=[keras.metrics.SparseCategoricalAccuracy()],)

In [9]:
gemma_lm.fit(data, epochs=2, batch_size=2)

Epoch 1/2
38/38 ━━━━━━━━━━━━━━━━━━━━ 3150s 83s/step - loss: 0.5048 - sparse_categorical_accuracy: 0.4773
Epoch 2/2
38/38 ━━━━━━━━━━━━━━━━━━━━ 3083s 81s/step - loss: 0.2655 - sparse_categorical_accuracy: 0.5898


In [10]:
gemma_lm.save("version_finetuned.keras")

In [11]:
%cd /kaggle/working
!ls

/kaggle/working


/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


version_finetuned.keras


In [12]:
from IPython.display import FileLink
FileLink(r'version_finetuned.keras')

/kaggle/working/version_finetuned.keras